# CNN Cats vs Dogs (Kaggle) — Estrutura com base única + CSV de labels

Este notebook trabalha com a seguinte estrutura:

```
base/          # pasta com milhares de fotos 
resultadosSelecao.csv    # duas colunas: id (nome do arquivo) e label (0=gato, 1=cachorro)
```

O pipeline inclui:
1. **Preparação dos dados**: leitura do CSV, divisão treino/val/test (80/10/10), pré-processamento, aumento de dados.
2. **Modelo CNN**: Conv2D + MaxPooling2D, Flatten + Dense, saída `sigmoid`.
3. **Treinamento**: 20 épocas, gráficos de acurácia e perda.
4. **Avaliação**: Precisão, Recall e F1-Score no conjunto de teste.
5. **Relatório**.

> Observação: ajuste os caminhos para sua estrutura real (por padrão: `base/images` e `base/labels.csv`).


In [2]:
# Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam


ModuleNotFoundError: No module named 'tensorflow'

In [4]:
# Configurações básicas
IMG_SIZE = (150, 150)   # você pode trocar para (224, 224) se preferir
BATCH_SIZE = 32
EPOCHS = 20

# Caminhos
BASE_DIR = 'base' # pasta com as imagens
CSV_PATH = 'resultadoSelecao.csv'  # arquivo CSV com id e label
print('IMAGES_DIR:', BASE_DIR)
print('CSV_PATH:', CSV_PATH)


IMAGES_DIR: base
CSV_PATH: resultadoSelecao.csv


In [ ]:
# Leitura do CSV e divisão em treino/val/test
# O CSV deve ter colunas: id (nome do arquivo, ex.: 'cat.0.jpg'), label (0=gato, 1=cachorro)
df = pd.read_csv(CSV_PATH)
print('Total de exemplos:', len(df))
print(df.head())

# Divide em treino (80%), validação (10%) e teste (10%)
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

print('Tamanhos -> treino:', len(train_df), ' | val:', len(val_df), ' | teste:', len(test_df))


In [ ]:
# Geradores com ImageDataGenerator usando flow_from_dataframe
# Pré-processamento: normalização + augment no treino
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# flow_from_dataframe requer colunas 'filename' e 'class' (podemos mapear)
train_df2 = train_df.copy()
val_df2 = val_df.copy()
test_df2 = test_df.copy()

# Adiciona caminho completo do arquivo
train_df2['filename'] = train_df2['id'].apply(lambda x: os.path.join(IMAGES_DIR, x))
val_df2['filename'] = val_df2['id'].apply(lambda x: os.path.join(IMAGES_DIR, x))
test_df2['filename'] = test_df2['id'].apply(lambda x: os.path.join(IMAGES_DIR, x))

# rótulos como string para binary class_mode
train_df2['class'] = train_df2['label'].astype(str)
val_df2['class'] = val_df2['label'].astype(str)
test_df2['class'] = test_df2['label'].astype(str)

train_gen = train_datagen.flow_from_dataframe(
    dataframe=train_df2,
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    color_mode='rgb',
    class_mode='binary',
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_gen = val_test_datagen.flow_from_dataframe(
    dataframe=val_df2,
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    color_mode='rgb',
    class_mode='binary',
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_gen = val_test_datagen.flow_from_dataframe(
    dataframe=test_df2,
    x_col='filename',
    y_col='class',
    target_size=IMG_SIZE,
    color_mode='rgb',
    class_mode='binary',
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [ ]:
# Construção da CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
# Treinamento (20 épocas)
history = model.fit(
    train_gen,
    steps_per_epoch=max(1, train_gen.samples // BATCH_SIZE),
    epochs=EPOCHS,
    validation_data=val_gen,
    validation_steps=max(1, val_gen.samples // BATCH_SIZE)
)


In [ ]:
# Gráficos de acurácia e perda
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia por época')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda por época')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Avaliação no conjunto de teste
loss, acc = model.evaluate(test_gen)
print(f'Acurácia no teste: {acc:.4f} | Perda: {loss:.4f}')

# Predições e métricas detalhadas
pred = model.predict(test_gen)
pred_classes = (pred > 0.5).astype(int).ravel()
true_classes = test_gen.classes

print('
Relatório de classificação:')
print(classification_report(true_classes, pred_classes, target_names=['Cat (0)', 'Dog (1)']))

precision = precision_score(true_classes, pred_classes)
recall = recall_score(true_classes, pred_classes)
f1 = f1_score(true_classes, pred_classes)
print(f'Precisão: {precision:.4f} | Recall: {recall:.4f} | F1-Score: {f1:.4f}')


# Conclusões

- **Pré-processamento**: Redimensionamento para 150x150, normalização (1/255) e aumento de dados (rotação, deslocamento, zoom, inversão horizontal).
- **Modelo**: CNN simples com 3 blocos Conv+Pool, Flatten e Dense (512), saída `sigmoid` para classificação binária.
- **Treinamento**: 20 épocas com Adam (lr=1e-4).
- **Avaliação**: métricas de acurácia, precisão, recall e F1-Score no conjunto de teste.

**Próximos passos**:
- Ajuste de hiperparâmetros (tamanho do batch, taxa de aprendizado, profundidade da rede).
- Regularização (Dropout, L2) para reduzir overfitting.
- **Transfer learning** (ex.: VGG16/ResNet50) para ganhos rápidos de performance.
